In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import plotly.express as px

from var import DATA_IN, DATA_OUT, START_DATE, END_DATE, FORECAST_HOURS_IN_ADVANCE
from src.io import read_time_series
from src.preprocess import resample_time_series

## TIDs catalog

In [ ]:
df_tid = read_time_series(
    Path(DATA_IN, 'TID_catalog.csv'),
    column_names=[
        'duration',
        'period',
        'amplitude',
        'spectral_contribution',
        'velocity',
        'azimuth',
        'quality_index',
        'datetime',
    ],
)

In [ ]:
df_tid_30 = resample_time_series(df_tid, aggregation_function='max')

Given the duration of a TID event, we repeat the information describing it for as long as it lasted

In [ ]:
df_tid_30['total_periods'] = (df_tid_30['duration'] * 60 / 30).round()

df_tid_30_ = pd.DataFrame()
for _, row in df_tid_30.dropna().reset_index().iterrows():
    periods = int(row['total_periods'])
    datetimes = pd.date_range(start=row['datetime'], periods=periods, freq='30T')
    df_tid_30_ = pd.concat(
        [df_tid_30_, pd.DataFrame({'datetime': datetimes, **row[1:]})],
        ignore_index=True,
    )
    
df_tid_30_ = df_tid_30_.set_index('datetime').drop(columns='total_periods')

## HF-EU index

In [ ]:
df_hf = read_time_series(
    Path(DATA_IN, 'HF_EU_IDX.csv'),
    column_names=['datetime', 'hf'],
).loc[START_DATE:END_DATE]

In [ ]:
df_hf_30 = resample_time_series(df_hf, aggregation_function='mean')
df_hf_30['hf'] = df_hf_30['hf'].round(2)

## Solar Data

In [ ]:
df_solar = read_time_series(
    Path(DATA_IN, 'solar_data.csv'),
    column_names=[
        'day_of_rotation',
        'n_sunsposts',
        'f_107_adj',
        'date',
    ],
    datetime_format="%d-%b-%Y"
).loc[START_DATE:END_DATE]

## Auroral Electrojet

In [ ]:
df_ejet = read_time_series(
    Path(DATA_IN, 'ImageIDX.csv'),
    column_names=['ie','datetime'],
    usecols=[2,3],
).loc[START_DATE:END_DATE]

In [ ]:
df_ejet_30 = resample_time_series(df_ejet, aggregation_function='median')
df_ejet_30['ie'] = df_ejet_30['ie'].round(1)

We detected **anomalous IE values**

The reported values appear to be affected by instrumental **offsets** that have a time window of 1 or more consecutive days

We removed the offset by subtracting a value equal to the average of the jumps before and after the affected day(s) (```ill_dates```)

In [ ]:
df_ejet_30['ie_diff'] = df_ejet_30['ie'].diff()

ill_dates = [
    ['2015-05-31', '2015-06-01'],
    ['2015-11-27', '2015-12-01'],
    ['2015-12-04', '2015-12-07'],
    ['2018-08-29', '2018-08-31'],
    ['2018-09-01', '2018-09-08'],
    ['2019-12-27', '2020-01-01'],
]

df_ejet_30['ie_fix'] = df_ejet_30['ie']
for range_ in ill_dates:
    offset_ = np.round(
        (
            df_ejet_30.loc[range_[0], 'ie_diff'].values[0] -
            df_ejet_30.loc[range_[-1], 'ie_diff'].values[0]
        ) / 2,
        1
    )
    
    df_ejet_30.loc[
        range_[0]: pd.to_datetime(range_[1]) - pd.Timedelta(minutes=30),
        'ie_fix',
    ] -= offset_
    
df_ejet_30['ie_fix'] = df_ejet_30['ie_fix'].clip(lower=0)
df_ejet_30 = df_ejet_30.drop(columns='ie_diff')

In [ ]:
#px.line(
#    data_frame=df_ejet_30.iloc[33_000:34_000],
#    y=['ie','ie_fix'],
#)

In [ ]:
df_ejet_30['ie_pct_change'] = df_ejet_30['ie_fix'].pct_change().round(3)

In [ ]:
#px.line(
#    data_frame=df_ejet_30.iloc[33_000: 34_000],
#    y='ie_pct_change',
#)

Construct IE moving averages with several rolling windows (1, 2, 3, 6, 12 and 24 hours)

In [ ]:
# n. of periods needed to get 3, 6, 12, 24 hours
periods = [2 * per_ for per_ in [3, 6, 12, 24]]

for per_ in periods:
    df_ejet_30[f'ie_mav_{per_/2:.0f}h'] = df_ejet_30['ie_fix'].rolling(
        window=int(per_)
    ).mean().round(1)
    
df_ejet_30 = df_ejet_30.drop(columns='ie').dropna()

## EDA

In [ ]:
# px.histogram(df_ejet_30['ie'])

In [ ]:
# px.histogram(df_tid['quality_index'])

In [ ]:
# px.histogram(df_tid['duration'])

In [ ]:
# px.bar(
#     data_frame=df_solar.groupby(
#         df_solar['date'].dt.year
#     )['f_107_adj'].mean().reset_index(),
#     x='date',
#     y='f_107_adj',
# )

In [ ]:
# px.histogram(df_solar[['f_107_adj','n_sunsposts']], barmode='overlay')

## Time serie analysis

Time distribution between one TID event and the following one

In [ ]:
periods = 16

second_order_diff = (
    24 * df_tid.index.to_series().diff(periods).diff(periods).dt.days
) + (
    df_tid.index.to_series().diff(periods).diff(periods).dt.total_seconds().div(3600)
).dropna()

In [ ]:
px.histogram(second_order_diff)

In [ ]:
second_order_diff.describe()

In [ ]:
second_order_diff.kurtosis(), second_order_diff.skew()

In [ ]:
from scipy.stats import shapiro, normaltest, kstest

shapiro(second_order_diff.dropna())

In [ ]:
normaltest(second_order_diff.dropna())

In [ ]:
# Kolmogorov-Smirnov test
ks_statistic, ks_p_value = kstest(second_order_diff.dropna(), 'cauchy')

print(f'KS Statistic: {ks_statistic}')
print(f'P-value: {ks_p_value}')

if ks_p_value < 0.05:
    print('Not a Cauchy-Lorentz distribution')
else:
    print('A Cauchy-Lorentz distribution')

## Dataset creation

In [ ]:
df_tid_30_.columns.to_list()

In [ ]:
df_j = df_ejet_30.merge(
    df_tid_30_['quality_index'],
    how='left',
    left_index=True,
    right_index=True,
).merge(
    df_hf_30,
    how='left',
    left_index=True,
    right_index=True,
).merge(
    df_solar[['n_sunsposts','f_107_adj']],
    how='left',
    left_index=True,
    right_index=True,
)

# Solar data need to be repeated, since they're provided on a daily basis only
df_j[['n_sunsposts','f_107_adj']] = df_j[['n_sunsposts','f_107_adj']].ffill()

# Include month of the year and hour of the day (UTC)
df_j['month'] = df_j.index.to_series().dt.month.astype(str).str.zfill(2)
df_j['hour'] = df_j.index.to_series().dt.hour.astype(str).str.zfill(2)

Construct the actual target, a boolean column which is set to `True` whenever a TID event is reported within a 3-hours timeframe

In [ ]:
steps = 2 * FORECAST_HOURS_IN_ADVANCE

df_j[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h'] = df_j['quality_index'].rolling(
    window=steps+1, min_periods=1
).sum().gt(0).shift(-steps, fill_value=False)

In [ ]:
# Class imbalance
100 * df_j[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h'].value_counts(normalize=True)

In [ ]:
# Good-quality events account for ~70% of the TID catalog
np.round(100 * df_tid['quality_index'].ge(0.5).sum() / df_tid.shape[0], 1)

In [ ]:
# (linear) correlations between variables
df_j.corr().style.map(
    lambda val: 'color: blue' if (val < -0.4 or val > 0.4) else 'color: black'
)

In [ ]:
px.histogram(
    df_j['ie_fix'].div(df_j['ie_mav_6h']).sample(frac=.6)
)

In [ ]:
px.histogram(
    df_j['ie_fix'].div(df_j['ie_mav_3h']).sample(frac=.6)
)

### Note to self!

if a feature is un-correlated with the target, it doesn't tell you that a **non-linear** model (*e.g.* CatBoost) wouldn't perform well by using this feature

Pearson correlation only takes into account *linear* correlation between variables; there might be non-linear (polynomial, logarithmic etc.) relationships between variables. Since the Pearson correlations are low, it seems that the relationships in the dataset (if any) might be non-linear and complex

$$\left( \frac{\rm{IE_0}}{\rm{IE_{3h}}} \right) ^ {\rm{HF}}$$

In [ ]:
(
    df_j['ie_fix'].div(
        df_j['ie_mav_3h']
    ) ** df_j['hf']
).corr(
    df_j[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h']
)

In [ ]:
px.histogram(
    (
        df_j['ie_fix'].div(
            df_j['ie_mav_3h']
        ) ** df_j['hf']
    ).sample(frac=.6)
)

$$\left( \frac{\rm{IE_0 - IE_{3h}}}{\rm{IE_{3h}}} \right) ^ {\rm{HF}}$$

In [ ]:
(
    (df_j['ie_fix'] - df_j['ie_mav_3h']).div(
        df_j['ie_mav_3h']
    ) ** df_j['hf']
).corr(
    df_j[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h']
)

In [ ]:
px.histogram(
    (
        (df_j['ie_fix'] - df_j['ie_mav_3h']).div(
            df_j['ie_mav_3h']
        ) ** df_j['hf']
    ).sample(frac=.6)
)

In [ ]:
df_tid.loc['2014-02-03']

In [ ]:
df_j.loc['2014-02-03 16': '2014-02-04 00']

Qui stiamo avendo il flag a True da 3h prima dell'inizio, fino alla fine dell'evento; vogliamo piuttosto solo il flag a True da 3h prima fino all'inizio dell'evento?